# LLM Evaluation with MLflow Example Notebook

In this notebook, we will demonstrate how to evaluate various LLMs and RAG systems with MLflow, leveraging simple metrics such as perplexity and toxicity, as well as LLM-judged metrics such as relevance, and even custom LLM-judged metrics such as professionalism

Set OpenAI Key

In [ ]:
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = "redacted"

In [ ]:
import openai
import pandas as pd

import mlflow

## Basic Question-Answering Evaluation

Create a test case of `inputs` that will be passed into the model and `ground_truth` which will be used to compare against the generated output from the model.

In [ ]:
eval_df = pd.DataFrame(
    {
        "inputs": [
            "How does useEffect() work?",
            "What does the static keyword in a function mean?",
            "What does the 'finally' block in Python do?",
            "What is the difference between multiprocessing and multithreading?",
        ],
        "ground_truth": [
            "The useEffect() hook tells React that your component needs to do something after render. React will remember the function you passed (we’ll refer to it as our “effect”), and call it later after performing the DOM updates.",
            "Static members belongs to the class, rather than a specific instance. This means that only one instance of a static member exists, even if you create multiple objects of the class, or if you don't create any. It will be shared by all objects.",
            "'Finally' defines a block of code to run when the try... except...else block is final. The finally block will be executed no matter if the try block raises an error or not.",
            "Multithreading refers to the ability of a processor to execute multiple threads concurrently, where each thread runs a process. Whereas multiprocessing refers to the ability of a system to run multiple processors in parallel, where each processor can run one or more threads.",
        ],
    }
)

Create a simple OpenAI model that asks gpt-3.5 to answer the question in two sentences. Call `mlflow.evaluate()` with the model and evaluation dataframe. 

In [ ]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question in two sentences"
    basic_qa_model = mlflow.openai.log_model(
        model="gpt-3.5-turbo",
        task=openai.ChatCompletion,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",  # specify which column corresponds to the expected output
        model_type="question-answering",  # model type indicates which metrics are relevant for this task
        evaluators="default",
    )
results.metrics

2023/10/19 22:35:53 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.7.2/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

2023/10/19 22:35:58 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/10/19 22:36:04 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: token_count
2023/10/19 22:36:04 INFO mlflow.metrics.metric_definitions: Computing token count metric:
2023/10/19 22:36:04 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: toxicity
2023/10/19 22:36:04 INFO mlflow.metrics.metric_definitions: Loading toxicity metric:
Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint
2023/10/19 22:36:05 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: RuntimeError("Failed to import transformers.models.roberta.modeling_tf_roberta because of the following error (look up to see its traceback):\nNo module named 'keras.engine'")), skipping metric logging.
2023/10/19 22:36:05 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: perplexity
2023/10/19 22:36

  0%|          | 0/1 [00:00<?, ?it/s]

2023/10/19 22:36:08 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: flesch_kincaid_grade_level
2023/10/19 22:36:08 INFO mlflow.metrics.metric_definitions: Computing flesch kincaid metric:
2023/10/19 22:36:08 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ari_grade_level
2023/10/19 22:36:08 INFO mlflow.metrics.metric_definitions: Computing automated readability index metric:
2023/10/19 22:36:08 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: exact_match


{'perplexity/v1/mean': 19.467615604400635,
 'perplexity/v1/variance': 18.95112384684103,
 'perplexity/v1/p90': 23.42769298553467,
 'flesch_kincaid_grade_level/v1/mean': 14.55,
 'flesch_kincaid_grade_level/v1/variance': 26.192499999999995,
 'flesch_kincaid_grade_level/v1/p90': 20.26,
 'ari_grade_level/v1/mean': 16.2,
 'ari_grade_level/v1/variance': 38.725,
 'ari_grade_level/v1/p90': 23.240000000000002,
 'exact_match/v1': 0.0}

Inspect the evaluation results table as a dataframe to see row-by-row metrics to further assess model performance

In [ ]:
results.tables["eval_results_table"]

inputs,ground_truth,outputs,token_count,perplexity/v1/score,flesch_kincaid_grade_level/v1/score,ari_grade_level/v1/score
How does useEffect() work?,"The useEffect() hook tells React that your component needs to do something after render. React will remember the function you passed (we’ll refer to it as our “effect”), and call it later after performing the DOM updates.",useEffect() is a hook in React that allows you to perform side effects in functional components. It takes a function as its first argument and runs that function after the component has rendered and whenever any of its dependencies have changed.,45,23.4797859192,11.3,12.4
What does the static keyword in a function mean?,"Static members belongs to the class, rather than a specific instance. This means that only one instance of a static member exists, even if you create multiple objects of the class, or if you don't create any. It will be shared by all objects.","The static keyword in a function means that the function can only be accessed within the same file it is declared in, and is not accessible to other files. It is used to limit the scope of the function and improve encapsulation.",46,18.20271492,10.3,10.0
What does the 'finally' block in Python do?,'Finally' defines a block of code to run when the try... except...else block is final. The finally block will be executed no matter if the try block raises an error or not.,"The 'finally' block in Python is used to define a block of code that will be executed regardless of whether an exception has occurred or not, ensuring that certain clean-up actions are always performed. It is typically used to close files, release resources, or clean up any operations that need to be done before exiting a try-except block.",68,23.306142807,13.4,16.1
What is the difference between multiprocessing and multithreading?,"Multithreading refers to the ability of a processor to execute multiple threads concurrently, where each thread runs a process. Whereas multiprocessing refers to the ability of a system to run multiple processors in parallel, where each processor can run one or more threads.","Multiprocessing involves the simultaneous execution of multiple tasks on multiple processors, while multithreading involves the execution of multiple threads within a single process, allowing for concurrent execution of different parts of the program.",39,12.8818187714,23.2,26.3


## LLM-judged correctness with OpenAI GPT-4

Construct an answer similarity metric using the `answer_similarity()` metric factory function.

In [ ]:
from mlflow.metrics.genai import EvaluationExample, answer_similarity

# Create an example to describe what answer_similarity means like for this problem.
example = EvaluationExample(
    input="What is MLflow?",
    output="MLflow is an open-source platform for managing machine "
    "learning workflows, including experiment tracking, model packaging, "
    "versioning, and deployment, simplifying the ML lifecycle.",
    score=4,
    justification="The definition effectively explains what MLflow is "
    "its purpose, and its developer. It could be more concise for a 5-score.",
    grading_context={
        "ground_truth": "MLflow is an open-source platform for managing "
        "the end-to-end machine learning (ML) lifecycle. It was developed by Databricks, "
        "a company that specializes in big data and machine learning solutions. MLflow is "
        "designed to address the challenges that data scientists and machine learning "
        "engineers face when developing, training, and deploying machine learning models."
    },
)

# Construct the metric using OpenAI GPT-4 as the judge
answer_similarity_metric = answer_similarity(model="openai:/gpt-4", examples=[example])

print(answer_similarity_metric)

EvaluationMetric(name=answer_similarity, greater_is_better=True, long_name=answer_similarity, version=v1, metric_details=
Task:
You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called answer_similarity based on the input and output.
A definition of answer_similarity and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before completing the task.

Input:
{input}

Output:
{output}

{grading_context_columns}

Metric definition:
Answer similarity is evaluated on the degree of semantic similarity of the provided output to the provided targets, which is the ground t

Call `mlflow.evaluate()` again but with your new `answer_similarity_metric`

In [ ]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[answer_similarity_metric],  # use the answer similarity metric created above
    )
results.metrics

2023/10/19 22:37:06 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/10/19 22:37:12 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: token_count
2023/10/19 22:37:12 INFO mlflow.metrics.metric_definitions: Computing token count metric:
2023/10/19 22:37:12 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: toxicity
2023/10/19 22:37:12 INFO mlflow.metrics.metric_definitions: Loading toxicity metric:
Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint
2023/10/19 22:37:13 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: RuntimeError("Failed to import transformers.models.roberta.modeling_tf_roberta because of the following error (look up to see its traceback):\nNo module named 'keras.engine'")), skipping metric logging.
2023/10/19 22:37:13 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: perplexity
2023/10/19 22:37

  0%|          | 0/1 [00:00<?, ?it/s]

2023/10/19 22:37:15 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: flesch_kincaid_grade_level
2023/10/19 22:37:15 INFO mlflow.metrics.metric_definitions: Computing flesch kincaid metric:
2023/10/19 22:37:15 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ari_grade_level
2023/10/19 22:37:15 INFO mlflow.metrics.metric_definitions: Computing automated readability index metric:
2023/10/19 22:37:15 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: exact_match
2023/10/19 22:37:15 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: answer_similarity


{'perplexity/v1/mean': 14.810191035270691,
 'perplexity/v1/variance': 55.66698687548323,
 'perplexity/v1/p90': 23.251440143585206,
 'flesch_kincaid_grade_level/v1/mean': 13.649999999999999,
 'flesch_kincaid_grade_level/v1/variance': 6.9025,
 'flesch_kincaid_grade_level/v1/p90': 16.4,
 'ari_grade_level/v1/mean': 16.05,
 'ari_grade_level/v1/variance': 9.427500000000002,
 'ari_grade_level/v1/p90': 19.32,
 'exact_match/v1': 0.0,
 'answer_similarity/v1/mean': 4.25,
 'answer_similarity/v1/variance': 0.1875,
 'answer_similarity/v1/p90': 4.7}

See the row-by-row LLM-judged answer similarity score and justifications

In [ ]:
results.tables["eval_results_table"]

inputs,ground_truth,outputs,token_count,perplexity/v1/score,flesch_kincaid_grade_level/v1/score,ari_grade_level/v1/score,answer_similarity/v1/score,answer_similarity/v1/justification
How does useEffect() work?,"The useEffect() hook tells React that your component needs to do something after render. React will remember the function you passed (we’ll refer to it as our “effect”), and call it later after performing the DOM updates.","useEffect() is a function in React that allows you to perform side effects in a functional component. It takes two arguments: a callback function that will be run after the component renders, and an array of dependencies to determine when the callback should be invoked.",51,15.5819368362,12.1,13.5,4,"The output provided by the model aligns well with the additional information provided. Both the output and the additional information explain that useEffect() is a function in React that performs actions after the component renders. The output also mentions the two arguments that useEffect() takes, which is not mentioned in the additional information. However, the output does not mention that React will remember the function passed to useEffect() and call it after performing the DOM updates, which is mentioned in the additional information. Therefore, while the output is largely accurate and aligns with the additional information in most aspects, it does not align in all significant aspects, which is why it receives a score of 4."
What does the static keyword in a function mean?,"Static members belongs to the class, rather than a specific instance. This means that only one instance of a static member exists, even if you create multiple objects of the class, or if you don't create any. It will be shared by all objects.","In C++, the static keyword used in a function declaration means that the function is limited to the scope of the translation unit and cannot be accessed by other files. In C#, the static keyword used in a function declaration means that the function belongs to the class itself and not to any specific instance of the class.",62,10.348818779,12.2,14.7,4,"The output provided by the model aligns well with the target information. It correctly explains the meaning of the static keyword in a function in both C++ and C#. However, it does not explicitly mention that only one instance of a static member exists, even if you create multiple objects of the class, or if you don't create any, which is mentioned in the target information. Therefore, while the output is largely accurate and informative, it lacks some details present in the target information, which prevents it from achieving a perfect score."
What does the 'finally' block in Python do?,'Finally' defines a block of code to run when the try... except...else block is final. The finally block will be executed no matter if the try block raises an error or not.,"The 'finally' block in Python is used to specify code that will always be executed, whether or not an exception is raised in the try block. This block is typically used for clean-up actions, such as closing files or releasing resources, to ensure they are always performed, regardless of any exceptions.",60,26.5383701324,12.1,14.7,5,"The output provided by the model aligns very closely with the provided targets. Both the output and the targets explain that the 'finally' block in Python is used to specify code that will always be executed, regardless of whether an exception is raised in the try block. The output also adds that this block is typically used for clean-up actions, which is an important aspect of the 'finally' block's functionality. Therefore, the output has a high degree of semantic similarity to the provided targets, warranting a score of 5."
What is the difference between multiprocessing and multithreading?,"Multithreading refers to the ability of a processor to execute multiple threads concurrently, where each thread runs a process. Whereas multiprocessing refers to the ab

## Custom LLM-judged metric for professionalism

Create a custom metric that will be used to determine professionalism of the model outputs. Use `make_genai_metric` with a metric definition, grading prompt, grading example, and judge model configuration

In [ ]:
from mlflow.metrics.genai import EvaluationExample, make_genai_metric

professionalism_metric = make_genai_metric(
    name="professionalism",
    definition=(
        "Professionalism refers to the use of a formal, respectful, and appropriate style of communication that is tailored to the context and audience. It often involves avoiding overly casual language, slang, or colloquialisms, and instead using clear, concise, and respectful language"
    ),
    grading_prompt=(
        "Professionalism: If the answer is written using a professional tone, below "
        "are the details for different scores: "
        "- Score 1: Language is extremely casual, informal, and may include slang or colloquialisms. Not suitable for professional contexts."
        "- Score 2: Language is casual but generally respectful and avoids strong informality or slang. Acceptable in some informal professional settings."
        "- Score 3: Language is balanced and avoids extreme informality or formality. Suitable for most professional contexts. "
        "- Score 4: Language is noticeably formal, respectful, and avoids casual elements. Appropriate for business or academic settings. "
        "- Score 5: Language is excessively formal, respectful, and avoids casual elements. Appropriate for the most formal settings such as textbooks. "
    ),
    examples=[
        EvaluationExample(
            input="What is MLflow?",
            output=(
                "MLflow is like your friendly neighborhood toolkit for managing your machine learning projects. It helps you track experiments, package your code and models, and collaborate with your team, making the whole ML workflow smoother. It's like your Swiss Army knife for machine learning!"
            ),
            score=2,
            justification=(
                "The response is written in a casual tone. It uses contractions, filler words such as 'like', and exclamation points, which make it sound less professional. "
            ),
        )
    ],
    version="v1",
    model="openai:/gpt-4",
    parameters={"temperature": 0.0},
    grading_context_columns=[],
    aggregations=["mean", "variance", "p90"],
    greater_is_better=True,
)

print(professionalism_metric)

EvaluationMetric(name=professionalism, greater_is_better=True, long_name=professionalism, version=v1, metric_details=
Task:
You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called professionalism based on the input and output.
A definition of professionalism and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before completing the task.

Input:
{input}

Output:
{output}

{grading_context_columns}

Metric definition:
Professionalism refers to the use of a formal, respectful, and appropriate style of communication that is tailored to the context and audience. I

TODO: Try out your new professionalism metric on a sample output to make sure it behaves as you expect

Call `mlflow.evaluate` with your new professionalism metric. 

In [ ]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],  # use the professionalism metric we created above
    )
print(results.metrics)

2023/10/19 22:41:22 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/10/19 22:41:27 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: token_count
2023/10/19 22:41:27 INFO mlflow.metrics.metric_definitions: Computing token count metric:
2023/10/19 22:41:27 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: toxicity
2023/10/19 22:41:27 INFO mlflow.metrics.metric_definitions: Loading toxicity metric:
Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint
2023/10/19 22:41:28 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: RuntimeError("Failed to import transformers.models.roberta.modeling_tf_roberta because of the following error (look up to see its traceback):\nNo module named 'keras.engine'")), skipping metric logging.
2023/10/19 22:41:28 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: perplexity
2023/10/19 22:41

  0%|          | 0/1 [00:00<?, ?it/s]

2023/10/19 22:41:31 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: flesch_kincaid_grade_level
2023/10/19 22:41:31 INFO mlflow.metrics.metric_definitions: Computing flesch kincaid metric:
2023/10/19 22:41:31 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ari_grade_level
2023/10/19 22:41:31 INFO mlflow.metrics.metric_definitions: Computing automated readability index metric:
2023/10/19 22:41:31 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: exact_match
2023/10/19 22:41:31 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: professionalism


{'perplexity/v1/mean': 19.43236994743347, 'perplexity/v1/variance': 5.924225461480717, 'perplexity/v1/p90': 21.905458068847658, 'flesch_kincaid_grade_level/v1/mean': 15.4, 'flesch_kincaid_grade_level/v1/variance': 28.564999999999998, 'flesch_kincaid_grade_level/v1/p90': 21.28, 'ari_grade_level/v1/mean': 18.625, 'ari_grade_level/v1/variance': 39.266875000000006, 'ari_grade_level/v1/p90': 25.520000000000003, 'professionalism/v1/mean': 4.0, 'professionalism/v1/variance': 0.0, 'professionalism/v1/p90': 4.0}


In [ ]:
results.tables["eval_results_table"]

inputs,ground_truth,outputs,token_count,perplexity/v1/score,flesch_kincaid_grade_level/v1/score,ari_grade_level/v1/score,professionalism/v1/score,professionalism/v1/justification
How does useEffect() work?,"The useEffect() hook tells React that your component needs to do something after render. React will remember the function you passed (we’ll refer to it as our “effect”), and call it later after performing the DOM updates.","useEffect() is a hook in React that allows you to perform side effects in functional components. It is triggered after the component is rendered and whenever the component's dependencies change, allowing you to run code at specific points during the component's lifecycle.",49,22.7291488647,12.7,15.0,4,"The language used in the output is formal and respectful. It avoids casual elements and provides a clear, concise explanation of the concept. It is appropriate for business or academic settings. However, it is not excessively formal to the point of being suitable for the most formal settings such as textbooks, hence it doesn't score a 5."
What does the static keyword in a function mean?,"Static members belongs to the class, rather than a specific instance. This means that only one instance of a static member exists, even if you create multiple objects of the class, or if you don't create any. It will be shared by all objects.","The ""static"" keyword in a function means that the function exists only within the scope of the current file, and cannot be called from other files. It also means that the function's local variables retain their values between multiple invocations of the function.",50,15.9082307816,10.5,13.5,4,"The language used in the output is formal and respectful. It avoids casual elements and is appropriate for business or academic settings. The explanation is clear and concise, making it easy to understand. The use of quotation marks around the term ""static"" shows a level of formality and precision. The language is not excessively formal, so it does not reach a score of 5, but it is more formal than a balanced language, which would score a 3. Therefore, a score of 4 is appropriate."
What does the 'finally' block in Python do?,'Finally' defines a block of code to run when the try... except...else block is final. The finally block will be executed no matter if the try block raises an error or not.,"In Python, the 'finally' block is used to execute a set of statements whether an exception is raised or not, ensuring that certain code is always executed regardless of the outcome of a try-except block. It is commonly used for activities such as closing files or releasing resources that need to be done regardless of whether an exception occurred or not.",69,19.9835128784,14.0,16.7,4,"The language used in the output is formal and respectful. It avoids casual elements and provides a clear, concise explanation of the 'finally' block in Python. The tone is appropriate for business or academic settings, hence the score of 4."
What is the difference between multiprocessing and multithreading?,"Multithreading refers to the ability of a processor to execute multiple threads concurrently, where each thread runs a process. Whereas multiprocessing refers to the ability of a system to run multiple processors in parallel, where each processor can run one or more threads.","Multiprocessing involves running multiple processes simultaneously on multiple cores to increase efficiency and achieve parallelism, while multithreading involves running multiple threads within a single process, allowing for concurrent execution and sharing of resources.",40,19.108587265,24.4,29.3,4,"The language used in the output is formal and respectful. It avoids casual elements and is appropriate for business or academic settings. The explanation is clear, concise, and uses technical terms appropriately. However, it is not excessively formal to the point of being suitable for the most formal settings such as textbooks, hence it does no

The professionalism score of the `basic_qa_model` is not very good. Let's try to create a new model that can perform better

Call `mlflow.evaluate()` using the new model. Observe that the professionalism score has increased!

In [ ]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question using extreme formality."
    professional_qa_model = mlflow.openai.log_model(
        model="gpt-3.5-turbo",
        task=openai.ChatCompletion,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        professional_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],
    )
print(results.metrics)

2023/10/19 22:41:57 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.7.2/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

2023/10/19 22:42:01 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/10/19 22:42:22 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: token_count
2023/10/19 22:42:22 INFO mlflow.metrics.metric_definitions: Computing token count metric:
2023/10/19 22:42:22 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: toxicity
2023/10/19 22:42:22 INFO mlflow.metrics.metric_definitions: Loading toxicity metric:
Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint
2023/10/19 22:42:22 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: RuntimeError("Failed to import transformers.models.roberta.modeling_tf_roberta because of the following error (look up to see its traceback):\nNo module named 'keras.engine'")), skipping metric logging.
2023/10/19 22:42:22 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: perplexity
2023/10/19 22:42

  0%|          | 0/1 [00:00<?, ?it/s]

2023/10/19 22:42:27 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: flesch_kincaid_grade_level
2023/10/19 22:42:27 INFO mlflow.metrics.metric_definitions: Computing flesch kincaid metric:
2023/10/19 22:42:27 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ari_grade_level
2023/10/19 22:42:27 INFO mlflow.metrics.metric_definitions: Computing automated readability index metric:
2023/10/19 22:42:27 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: exact_match
2023/10/19 22:42:27 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: professionalism


{'perplexity/v1/mean': 25.524279594421387, 'perplexity/v1/variance': 72.94819473072403, 'perplexity/v1/p90': 32.810652542114255, 'flesch_kincaid_grade_level/v1/mean': 16.525, 'flesch_kincaid_grade_level/v1/variance': 0.7818749999999987, 'flesch_kincaid_grade_level/v1/p90': 17.509999999999998, 'ari_grade_level/v1/mean': 19.125, 'ari_grade_level/v1/variance': 1.056875000000001, 'ari_grade_level/v1/p90': 20.23, 'professionalism/v1/mean': 4.5, 'professionalism/v1/variance': 0.25, 'professionalism/v1/p90': 5.0}


In [ ]:
results.tables["eval_results_table"]

inputs,ground_truth,outputs,token_count,perplexity/v1/score,flesch_kincaid_grade_level/v1/score,ari_grade_level/v1/score,professionalism/v1/score,professionalism/v1/justification
How does useEffect() work?,"The useEffect() hook tells React that your component needs to do something after render. React will remember the function you passed (we’ll refer to it as our “effect”), and call it later after performing the DOM updates.","I would be delighted to expound upon the mechanics of the useEffect() function. useEffect() is a hook provided by the React library that facilitates the synchronization of side effects with the React components lifecycle method. It serves as a tool for performing actions that necessitate interaction with external entities, such as fetching data from an API or subscribing to events. The principal construct of useEffect() hinges on the concept of dependencies. By specifying an array of dependencies, which can consist of state variables, the useEffect() hook will execute the specified effect only when these dependencies undergo changes. Once the component mounts, useEffect() initially triggers the specified effect, consequently performing the designated side effect. During subsequent renders, useEffect() will carry out the cleanup operation, accommodating for the possibility of multiple effects being executed successively. Should any of the supplied dependencies undergo alteration, useEffect() will then reexecute the effect, ensuring synchronization with the altered state of the component. In essence, useEffect() effectively accommodates for the execution of side effects with accuracy and efficiency, aligning them with the React component lifecycle and facilitating the necessary synchronization with state updates.",220,25.9580955505,16.1,18.0,4,"The language used in the output is formal and respectful, with no casual elements or slang. The explanation is clear, concise, and detailed, demonstrating a high level of professionalism. However, it does not reach the level of excessive formality that would be expected in the most formal settings such as textbooks, hence the score is not 5."
What does the static keyword in a function mean?,"Static members belongs to the class, rather than a specific instance. This means that only one instance of a static member exists, even if you create multiple objects of the class, or if you don't create any. It will be shared by all objects.","The static keyword, when applied to a function within the realm of computer programming, denotes a special attribute that imparts a significant alteration to the behavior of the said function. Specifically, the static keyword, operating in this context, restricts the visibility and scope of the function to the current translation unit, consequently prohibiting its accessibility from other external units of code. Consequently, the function annotated with the static modifier may solely be invoked within the same translation unit, and any attempts to access or execute it from an external module shall be rendered futile.",107,31.0698623657,17.9,20.8,5,"The language used in the output is excessively formal and respectful. It avoids casual elements and is appropriate for the most formal settings such as textbooks. The response is clear, concise, and uses technical language appropriate for the subject matter. It avoids any slang or colloquialisms, and is tailored to an audience that understands the context of the question."
What does the 'finally' block in Python do?,'Finally' defines a block of code to run when the try... except...else block is final. The finally block will be executed no matter if the try block raises an error or not.,"The 'finally' block in Python encompasses a set of statements that are executed regardless of whether an exception occurred or not in the preceding 'try' and 'except' blocks. It serves as a reliable means to perform essential tasks, such as closing files or releasing resources, in order to uphold code integrity 